In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os
import errno
import string
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score
from sklearn.metrics import accuracy_score,roc_auc_score,mean_absolute_error,cohen_kappa_score
from sklearn.preprocessing import label_binarize,LabelEncoder
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold,StratifiedKFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy,mode
from scipy.io import arff
from sklearn.metrics import pairwise_distances
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import load_iris
import warnings
import random
from statistics import mode
random.seed(0)
warnings.filterwarnings('ignore')

def loadNonIRDatasets(dataset):
    documents,labels=[],[]
    fname=dataset+'.csv'
    with open(fname, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
        for row in reader:
            if len(row)>0:
                if '?' in row:
                    continue
                doc=row[0:len(row)-2]
                try:
                    doc=[abs(float(d)) for d in doc]
                except:
                    continue
                documents.append(doc)
                #print(row)
                labels.append(row[len(row)-1])

    return documents,labels

def Cosine(a, b):#distance
    return distance.cosine(a,b)

def Euclidean(a, b):#distance
    return distance.euclidean(a,b)

def most_common(lst):
    return max(set(lst), key=lst.count)
def PrintDetails(metric="smtp",measure= "Accuracy",Arr=None,kList=[5,15,30,45],train_time=0,test_time=0,std=[]):
    x = PrettyTable()
    #kList=[1,3,5,9,15,30,45]#,70,90,120]
    if std==[]:
        x.field_names = ["k",measure,"Train Time","Test time","Total Time"]
    else:
        x.field_names = ["Split",measure,"std","Train Time","Test time","Total Time"]
        
    print("metric",metric)
    print("measure",measure)
    tables.write(("metric\t"+str(metric)+"\n").encode())
    tables.write(("measure\t"+str(measure)+"\n").encode())
    average=0
    total_time=np.add(train_time,test_time)
    for i in range(0,len(Arr)):
        print(train_time[i])
        tr_time=str(timedelta(seconds=(train_time[i])))
        ts_time=str(timedelta(seconds=(test_time[i])))
        
        tot_time=str(timedelta(seconds=(total_time[i])))
        if std==[]:
            x.add_row([str(kList[i]),round(Arr[i],4),tr_time,ts_time,tot_time]) 
        else:
            x.add_row([str(kList[i]),round(Arr[i],4),round(std[i],4),tr_time,ts_time,tot_time]) 
            
        average+=Arr[i]
    if std==[]:
        x.add_row(["Average",round((average/len(Arr)),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),np.std(train_time),
                   np.std(test_time),np.std(total_time)])   
    else:
        x.add_row(["Average",round((average/len(Arr)),4),round(np.mean(std),4),str(timedelta(seconds=np.mean(train_time))),
                   str(timedelta(seconds=np.mean(test_time))),str(timedelta(seconds=np.mean(total_time)))]) 
        x.add_row(["Std Dev.",round(np.std(Arr),4),round(np.std(std),4),
                   round(np.std(train_time),10),
                   round(np.std(test_time),10),
                   round(np.std(total_time),10)])
        
    x.padding_width =1
    tables.write(str(x).encode())
    print(x)

def findDI(train_data,train_labels,metric,k):
    clss=set(train_labels)
    func=globals()[metric]
    radius={}
    th=0.5
    for cl in clss:
        indices=[i for i in range(0,len(train_labels)) if train_labels[i]==cl]
        elements=train_data[indices,:]
        distMat=pairwise_distances(elements,elements,metric=func)
        radius[cl]=np.sum(distMat)/(distMat.shape[0]+distMat.shape[1])
    weights=[]
   
    classifier = NearestNeighbors(n_neighbors=k,metric=func)
    if k>len(train_labels):
         classifier = NearestNeighbors(n_neighbors=len(train_labels),metric=func)
        
    classifier.fit(train_data,train_labels)
    distances,neighVals=classifier.kneighbors(train_data,return_distance=True)
    print(neighVals.shape)
    for i in range(0,train_data.shape[0]):
        r=radius[train_labels[i]]
        ID=0
        neighs=np.array(neighVals[i,:],dtype=int)
        #print(neighs)
        neighLabels = [train_labels[i] for i in neighs]
           
        #neighLabels = [train_labels[i] for i in int(neighVals[i,:])]
        val=0
        c=1
        for nn,neigh in enumerate(neighLabels):
            if distances[i,nn]<r:
                c=c+1
                if neigh==train_labels[i]:
                    val=val+1
        ID=(val/c)
        ##############find weights using validity ###############
        #w=np.sum(val*(1/(distances[i]+alpha)))
        weights.append(ID)
    return weights

def classify(k=3,metric="euclidean",termOccurance=None, docOccurance=None):
    accuracy=[]
    precision=[]
    Recall=[]
    fMeasure=[]
    mprecision=[]
    mRecall=[]
    mfMeasure=[]
    roc=[]
    y_pred=[]
    mae=[]
    kappa=[]
    averageMeanPrecison=[]
    func=globals()[metric]
    kList=[5,15,30,45]
    theta=1
    var=np.zeros(train_data.shape[1])
    train_time=[]
    test_time=[]
    M=5
    alpha=0.5
    for k in kList:
        time1=time.time()
        weights=findDI(train_data,train_labels,metric,k)
        classifier = KNeighborsClassifier(n_neighbors=k,metric=func)
        if k>len(train_labels):
             classifier = KNeighborsClassifier(n_neighbors=len(train_labels),metric=func)
        classifier.fit(train_data,train_labels)
        distances,neighVals=classifier.kneighbors(test_data,return_distance=True)
        pred=[]
        time2=time.time()
        for i in range(0,test_data.shape[0]):
            neighs=np.array(neighVals[i,:],dtype=int)
            neighLabels = [train_labels[i] for i in neighs]
            dLabels=set(neighLabels)
            weightSum=0
            maxweight=0
            predLabel=''
            for lbl in dLabels:
                weightSum=np.mean([weights[ii]/(distances[i][j]+0.00001) for j,ii in enumerate(neighs) if train_labels[ii]==lbl])
                if weightSum>=maxweight:
                    maxweight=weightSum
                    predLabel=lbl
            pred.append(predLabel)  
              
        y_pred.append(pred)
        time3=time.time()
        train_time.append(time2-time1)
        test_time.append(time3-time2)
 
   
    class_dict={}
    classes=list(set(labels))
    for i in range(0,len(classes)):
        class_dict[classes[i]]=i
    print(class_dict)
    num_testLabels=[]
    for i in range(0,len(test_labels)):
        num_testLabels.append(class_dict[test_labels[i]])
       
    for i in range(0,len(y_pred)):
        test_labels1=list(test_labels)
        #Accuracy
       
        accuracy.append(accuracy_score(test_labels, y_pred[i]))
        
        #Precison
        precision.append(precision_score(test_labels, y_pred[i],average='macro'))
         
        #Recall
        Recall.append(recall_score(test_labels, y_pred[i],average='macro'))
           
        #f measure
        fMeasure.append(f1_score(test_labels, y_pred[i],average='macro'))
        
        #Precison
        mprecision.append(precision_score(test_labels, y_pred[i],average='micro'))
         
        #Recall
        mRecall.append(recall_score(test_labels, y_pred[i],average='micro'))
           
        #f measure
        mfMeasure.append(f1_score(test_labels, y_pred[i],average='micro'))
        
        kappa.append(cohen_kappa_score(test_labels, y_pred[i]))
      
        test = label_binarize(test_labels1, classes=categories).reshape((-1))
        pred = label_binarize( y_pred[i], classes=categories).reshape((-1))
        
        #roc
        try:
            roc.append(roc_auc_score(test,pred))
        except:
            roc.append(-1)
            
        #Average Mean Precision
        #y_val_true, val_pred = y_val_true.reshape((-1)), val_pred.reshape((-1))
        averageMeanPrecison.append(average_precision_score(test, pred))
        #MAE
        num_pred=[]
        for j in range(0,len(y_pred[i])):
            num_pred.append(class_dict[y_pred[i][j]])
        mae.append(mean_absolute_error(num_testLabels, num_pred))
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="Average Mean Precison",Arr=averageMeanPrecison)


    ''''''
   #Accuracy
    PrintDetails(metric=metric,measure="Accuracy",Arr=accuracy,train_time=train_time,test_time=test_time)
    
    #Precison
    PrintDetails(metric=metric,measure="Precision",Arr=precision,train_time=train_time,test_time=test_time)
    #Precison
    PrintDetails(metric=metric,measure="micro Precision",Arr=mprecision,train_time=train_time,test_time=test_time)
    #Recall
    PrintDetails(metric=metric,measure="Recall",Arr=Recall,train_time=train_time,test_time=test_time)
     #Recall
    PrintDetails(metric=metric,measure="micro Recall",Arr=mRecall,train_time=train_time,test_time=test_time)
    #f measure
    PrintDetails(metric=metric,measure="F Measure",Arr=fMeasure,train_time=train_time,test_time=test_time)
    
    #f measure
    PrintDetails(metric=metric,measure="micro F Measure",Arr=mfMeasure,train_time=train_time,test_time=test_time)
    
    #roc
    PrintDetails(metric=metric,measure="ROC",Arr=roc,train_time=train_time,test_time=test_time)
    #Average Mean Precision
    PrintDetails(metric=metric,measure="Average Mean Precison",Arr=averageMeanPrecison,train_time=train_time,test_time=test_time)
    #MAE
    PrintDetails(metric=metric,measure="MAE",Arr=mae,train_time=train_time,test_time=test_time)
  
    accuracy_avg.append(accuracy)
    precision_avg.append(precision)
    recall_avg.append(Recall)
    macroFMeasure_avg.append(fMeasure)
    mprecision_avg.append(mprecision)
    mrecall_avg.append(mRecall)
    mFMeasure_avg.append(mfMeasure)
    roc_avg.append(roc)
    AMP_avg.append(averageMeanPrecison)
    mae_avg.append(mae)
    avg_test_time.append(test_time)
    avg_train_time.append(train_time)
    
import glob
datasets=[]
for file_name in glob.glob("Datasets April\\"+'*.csv'):
    file=os.path.split(file_name)[1]
    datasets.append(os.path.splitext(file)[0])

    
for dataset in datasets:
    fname="Datasets April\\"+dataset
    print(fname.encode())
    arr,labels=loadNonIRDatasets(fname)
    arr=np.array(arr)
    labels=np.array(labels)
    labels=list(labels)
    le = LabelEncoder()
    labels=le.fit_transform(labels)
    categories=list(set(labels))
    measures=["Euclidean"]
    global w
    w=0
    global Weights
    for met in measures:
        n_split=1
        labels=np.array(labels)
        path = "PrkNN\\Cancelled Datasets"
        if not os.path.exists(path):
            os.makedirs(path)
        fname=path+'\\table_'+dataset+'_'+met+'_PrKNN.txt'
        tables = open(fname, 'wb')
        k_splits=10
        accuracy_avg=[]
        precision_avg=[]
        recall_avg=[]
        macroFMeasure_avg=[]
        mprecision_avg=[]
        mrecall_avg=[]
        mFMeasure_avg=[]
        roc_avg=[]
        AMP_avg=[]
        mae_avg=[]
        avg_test_time=[]
        avg_train_time=[]
        kf = StratifiedKFold(n_splits=k_splits)
        for trn_ind, tst_ind in kf.split(arr,labels):
            xTrain, xTest = arr[trn_ind], arr[tst_ind]
            yTrain, yTest = labels[trn_ind], labels[tst_ind]
            train_data=np.array(xTrain)
            test_data=np.array(xTest)
            train_labels=np.array(yTrain)
            test_labels=np.array(yTest)
            allData=arr
            termOcc=[]
            docOcc=[]
            tables.write("\n*************".encode())
            tables.write((" Split\t"+str(n_split)).encode())
            tables.write(" ***********\n".encode())
            n_split=n_split+1
            k=1
            print("###############")
            Weights=np.ones(train_data.shape[0])
            classify(k=k,metric=met,termOccurance=termOcc,docOccurance=docOcc)
        list1=[5,15,30,45]
        tables.write("\n************* Average Results ***********\n".encode())
        acc_std=np.std(np.array(accuracy_avg),axis=0)
        accuracy_avg=np.mean(np.array(accuracy_avg),axis=0)
        
        pr_std=np.std(np.array(precision_avg),axis=0)
        precision_avg=np.mean(np.array(precision_avg),axis=0)
        
        mpr_std=np.std(np.array(mprecision_avg),axis=0)
        mprecision_avg=np.mean(np.array(mprecision_avg),axis=0)
        
        rcl_std=np.std(np.array(recall_avg),axis=0)
        recall_avg=np.mean(np.array(recall_avg),axis=0)
        
        mrcl_std=np.std(np.array(mrecall_avg),axis=0)
        mrecall_avg=np.mean(np.array(mrecall_avg),axis=0)
        
        F_std=np.std(np.array(macroFMeasure_avg),axis=0)
        macroFMeasure_avg=np.mean(np.array(macroFMeasure_avg),axis=0)
        
        mF_std=np.std(np.array(mFMeasure_avg),axis=0)
        mFMeasure_avg=np.mean(np.array(mFMeasure_avg),axis=0)
        
        roc_std=np.std(np.array(roc_avg),axis=0)
        roc_avg=np.mean(np.array(roc_avg),axis=0)
        
        amp_std=np.std(np.array(AMP_avg),axis=0)
        AMP_avg=np.mean(np.array(AMP_avg),axis=0)
        
        mae_std=np.std(np.array(mae_avg),axis=0)
        mae_avg=np.mean(np.array(mae_avg),axis=0)
        
        avg_train_time=np.mean(np.array(avg_train_time),axis=0)
        avg_test_time=np.mean(np.array(avg_test_time),axis=0)
        #Accuracy
        PrintDetails(metric=met,measure="Accuracy",Arr=accuracy_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=acc_std)
        
        #Precision
        PrintDetails(metric=met,measure="Precision",Arr=precision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=pr_std)
        
        #Precision
        PrintDetails(metric=met,measure="Micro Precision",Arr=mprecision_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mpr_std)


        #Recall
        PrintDetails(metric=met,measure="Recall",Arr=recall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=rcl_std)
        
         #Recall
        PrintDetails(metric=met,measure="Micro Recall",Arr=mrecall_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mrcl_std)


        #f measure
        PrintDetails(metric=met,measure="F Measure",Arr=macroFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=F_std)
        
       
       
        #f measure
        PrintDetails(metric=met,measure="Micro F Measure",Arr=mFMeasure_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mF_std)


        #roc
        PrintDetails(metric=met,measure="ROC",Arr=roc_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=roc_std)

        #average Mean precision
        PrintDetails(metric=met,measure="AMP",Arr=AMP_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=amp_std)
         
        #MAE
        PrintDetails(metric=met,measure="MAE",Arr=mae_avg,kList=list1,train_time=avg_train_time,test_time=avg_test_time,std=mae_std)
        
        tables.close()
        print("###############")





b'Datasets April\\1_lung-cancer'
###############
(27, 5)
(27, 15)
(27, 27)
(27, 27)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
0.0069997310638427734
0.00599980354309082
0.006000041961669922
0.0060002803802490234
+----------+----------+-----------------------+-----------+-----------------------+
|    k     | Accuracy |       Train Time      | Test time |       Total Time      |
+----------+----------+-----------------------+-----------+-----------------------+
|    5     |   0.75   |     0:00:00.007000    |  0:00:00  |     0:00:00.007000    |
|    15    |   0.75   |     0:00:00.006000    |  0:00:00  |     0:00:00.006000    |
|    30    |   0.75   |     0:00:00.006000    |  0:00:00  |     0:00:00.006000    |
|    45    |   0.75   |     0:00:00.006000    |  0:00:00  |     0:00:00.006000    |
| Average  |   0.75   |     0:00:00.006250    |  0:00:00  |     0:00:00.006250    |
| Std Dev. |   0.0    | 0.0004328781120128417 |    0.0    | 0.0004328781120128417 |
+----------+----------+-----

metric Euclidean
measure Accuracy
0.006000041961669922
0.005999326705932617
0.00800013542175293
0.0069997310638427734
+----------+----------+-----------------------+-----------------------+-----------------------+
|    k     | Accuracy |       Train Time      |       Test time       |       Total Time      |
+----------+----------+-----------------------+-----------------------+-----------------------+
|    5     |  0.6667  |     0:00:00.006000    |        0:00:00        |     0:00:00.006000    |
|    15    |  0.6667  |     0:00:00.005999    |     0:00:00.001000    |     0:00:00.007000    |
|    30    |  0.6667  |     0:00:00.008000    |        0:00:00        |     0:00:00.008000    |
|    45    |  0.6667  |     0:00:00.007000    |     0:00:00.001000    |     0:00:00.008000    |
| Average  |  0.6667  |     0:00:00.006750    |     0:00:00.000500    |     0:00:00.007250    |
| Std Dev. |   0.0    | 0.0008293297722262893 | 0.0005000829980825205 | 0.0008291140958201877 |
+----------+------

metric Euclidean
measure Accuracy
0.0060002803802490234
0.006002902984619141
0.006035566329956055
0.005997657775878906
+----------+----------+------------------------+-----------------------+-----------------------+
|    k     | Accuracy |       Train Time       |       Test time       |       Total Time      |
+----------+----------+------------------------+-----------------------+-----------------------+
|    5     |  0.6667  |     0:00:00.006000     |        0:00:00        |     0:00:00.006000    |
|    15    |  0.6667  |     0:00:00.006003     |     0:00:00.000998    |     0:00:00.007001    |
|    30    |  0.6667  |     0:00:00.006036     |     0:00:00.000998    |     0:00:00.007033    |
|    45    |  0.6667  |     0:00:00.005998     |        0:00:00        |     0:00:00.005998    |
| Average  |  0.6667  |     0:00:00.006009     |     0:00:00.000499    |     0:00:00.006508    |
| Std Dev. |   0.0    | 1.5391392013148694e-05 | 0.0004988909052549146 | 0.0005091516955752079 |
+-------

+----------+--------+-----------------------+----------------------+-----------------------+
|    k     |  MAE   |       Train Time      |      Test time       |       Total Time      |
+----------+--------+-----------------------+----------------------+-----------------------+
|    5     | 0.3333 |     0:00:00.009001    |       0:00:00        |     0:00:00.009001    |
|    15    | 0.3333 |     0:00:00.008000    |       0:00:00        |     0:00:00.008000    |
|    30    | 0.3333 |     0:00:00.007999    |    0:00:00.001002    |     0:00:00.009001    |
|    45    | 0.3333 |     0:00:00.012000    |    0:00:00.001000    |     0:00:00.013000    |
| Average  | 0.3333 |     0:00:00.009250    |    0:00:00.000500    |     0:00:00.009750    |
| Std Dev. |  0.0   | 0.0016393592876822048 | 0.000500441051880956 | 0.0019198602260249462 |
+----------+--------+-----------------------+----------------------+-----------------------+
###############
(28, 5)
(28, 15)
(28, 28)
(28, 28)
{0: 0, 1: 1}
metric

###############
(405, 5)
(405, 15)
(405, 30)
(405, 45)
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12}
metric Euclidean
measure Accuracy
0.9609978199005127
0.9140024185180664
1.0389981269836426
0.9669978618621826
+----------+----------+---------------------+-----------------------+---------------------+
|    k     | Accuracy |      Train Time     |       Test time       |      Total Time     |
+----------+----------+---------------------+-----------------------+---------------------+
|    5     |  0.587   |    0:00:00.960998   |     0:00:00.002000    |    0:00:00.962998   |
|    15    |  0.5217  |    0:00:00.914002   |     0:00:00.003000    |    0:00:00.917003   |
|    30    |  0.5217  |    0:00:01.038998   |     0:00:00.006001    |    0:00:01.044999   |
|    45    |  0.5217  |    0:00:00.966998   |     0:00:00.007001    |    0:00:00.973999   |
| Average  |  0.538   |    0:00:00.970249   |     0:00:00.004501    |    0:00:00.974750   |
| Std Dev. |  

...

+----------+-----------------------+---------------------+-----------------------+--------------------+
|    k     | Average Mean Precison |      Train Time     |       Test time       |     Total Time     |
+----------+-----------------------+---------------------+-----------------------+--------------------+
|    5     |         0.6975        |    0:00:01.375033   |     0:00:00.002001    |   0:00:01.377033   |
|    15    |         0.6975        |    0:00:01.484964   |     0:00:00.002000    |   0:00:01.486964   |
|    30    |         0.6975        |    0:00:01.718004   |     0:00:00.002999    |   0:00:01.721002   |
|    45    |         0.7258        |    0:00:02.137000   |     0:00:00.006001    |   0:00:02.143001   |
| Average  |         0.7046        |    0:00:01.678750   |     0:00:00.003250    |   0:00:01.682000   |
| Std Dev. |         0.0122        | 0.29211751865980407 | 0.0016397500200712736 | 0.2937202757195371 |
+----------+-----------------------+---------------------+------

+----------+-----------------+---------------------+-----------------------+--------------------+
|    k     | micro F Measure |      Train Time     |       Test time       |     Total Time     |
+----------+-----------------+---------------------+-----------------------+--------------------+
|    5     |      0.8909     |    0:00:01.385968   |     0:00:00.001002    |   0:00:01.386970   |
|    15    |      0.8545     |    0:00:01.503997   |     0:00:00.002001    |   0:00:01.505998   |
|    30    |      0.9636     |    0:00:01.789999   |     0:00:00.005000    |   0:00:01.794999   |
|    45    |      0.8545     |    0:00:01.947000   |     0:00:00.003999    |   0:00:01.950999   |
| Average  |      0.8909     |    0:00:01.656741   |     0:00:00.003000    |   0:00:01.659741   |
| Std Dev. |      0.0445     | 0.22285473965673513 | 0.0015802703509052312 | 0.2242584115473707 |
+----------+-----------------+---------------------+-----------------------+--------------------+
metric Euclidean
mea

(499, 5)
(499, 15)
(499, 30)
(499, 45)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.422004222869873
1.5839645862579346
1.8849902153015137
1.9819998741149902
+----------+----------+---------------------+----------------------+--------------------+
|    k     | Accuracy |      Train Time     |      Test time       |     Total Time     |
+----------+----------+---------------------+----------------------+--------------------+
|    5     |  0.8545  |    0:00:01.422004   |    0:00:00.002033    |   0:00:01.424038   |
|    15    |  0.8545  |    0:00:01.583965   |    0:00:00.002009    |   0:00:01.585973   |
|    30    |  0.9273  |    0:00:01.884990   |    0:00:00.003000    |   0:00:01.887990   |
|    45    |  0.8909  |    0:00:01.982000   |    0:00:00.004000    |   0:00:01.986000   |
| Average  |  0.8818  |    0:00:01.718240   |    0:00:00.002761    |   0:00:01.721000   |
| Std Dev. |  0.0302  | 0.22536660571358308 | 0.000819666249209034 | 0.2261246754527767 |
+----------+----------+------

(499, 5)
(499, 15)
(499, 30)
(499, 45)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.44100022315979
1.5530009269714355
1.9549999237060547
1.9540002346038818
+----------+----------+---------------------+-----------------------+---------------------+
|    k     | Accuracy |      Train Time     |       Test time       |      Total Time     |
+----------+----------+---------------------+-----------------------+---------------------+
|    5     |  0.7455  |    0:00:01.441000   |     0:00:00.001000    |    0:00:01.442000   |
|    15    |  0.8545  |    0:00:01.553001   |     0:00:00.002001    |    0:00:01.555002   |
|    30    |  0.8909  |    0:00:01.955000   |     0:00:00.003000    |    0:00:01.958000   |
|    45    |  0.9636  |    0:00:01.954000   |     0:00:00.004000    |    0:00:01.958000   |
| Average  |  0.8636  |    0:00:01.725750   |     0:00:00.002500    |    0:00:01.728251   |
| Std Dev. |  0.0787  | 0.23215209171915113 | 0.0011179797547666303 | 0.23319749381109203 |
+----------+

###############
(499, 5)
(499, 15)
(499, 30)
(499, 45)
{0: 0, 1: 1}
metric Euclidean
measure Accuracy
1.4350004196166992
1.6070001125335693
1.896010398864746
2.092003345489502
+----------+----------+--------------------+-----------------------+---------------------+
|    k     | Accuracy |     Train Time     |       Test time       |      Total Time     |
+----------+----------+--------------------+-----------------------+---------------------+
|    5     |  0.9636  |   0:00:01.435000   |     0:00:00.001000    |    0:00:01.436001   |
|    15    |  0.8182  |   0:00:01.607000   |     0:00:00.003000    |    0:00:01.610000   |
|    30    |  0.9273  |   0:00:01.896010   |     0:00:00.004990    |    0:00:01.901000   |
|    45    |  0.8545  |   0:00:02.092003   |     0:00:00.006998    |    0:00:02.099001   |
| Average  |  0.8909  |   0:00:01.757504   |     0:00:00.003997    |    0:00:01.761501   |
| Std Dev. |  0.0575  | 0.2538374549594676 | 0.0022340433089183586 | 0.25606113466199715 |
+----

In [2]:
"test"

'test'